In [ ]:
def get_randoms(std_devs):
    """
    根据给定的标准差列表生成随机数列表。

    参数:
    std_devs (list): 包含每个随机数标准差的列表

    返回:
    list: 包含生成的随机数的列表，按以下顺序排列：
        - 小麦预期销售量增长率，介于0.05~0.1之间
        - 玉米预期销售量增长率，介于0.05~0.1之间
        - 其他农作物预期销售量相对于2023年的变化率，介于-0.05~+0.05之间
        - 农作物亩产量每年的变化率，介于-0.1~+0.1之间
        - 农作物种植成本每年的增长率，在0.05附近
        - 粮食类作物销售价格每年的增长率，在0附近
        - 蔬菜类作物销售价格每年的增长率，在0.05附近
        - 食用菌的销售价格每年的增长率，在-0.05~-0.01之间
        - 羊肚菌的销售价格每年下降幅度为5%
    """
    # 定义均值
    means = [
        (0.05 + 0.1) / 2,  # es_wheet_growth_mean
        (0.05 + 0.1) / 2, # es_corn_growth_mean
        (-0.05 + 0.05) / 2,  # es_others_rate_mean
        (-0.1 + 0.1) / 2,  # ty_rate_mean
        0.05,  # pc_growth_mean
        0,  # sp_growth_mean
        0.05,  # vp_growth_mean
        (-0.05 + -0.01) / 2  # mp_growth_mean
    ]
    
    # 生成随机数
    randoms = [np.random.normal(loc=mean, scale=std) for mean, std in zip(means, std_devs)]
    
    # 添加固定值
    randoms.append(-0.05)  # m1p_growth
    
    return randoms

In [ ]:

# 打印结果
print('小麦的预期销售量增长率：', es_wheet_growth)
print('玉米的预期销售量增长率：', es_corn_growth)
print('其他农作物预期销售量相对于2023年的变化率：', es_others_rate)
print('农作物亩产量每年的变化率：', ty_rate)
print('农作物种植成本每年的增长率：', pc_growth)
print('粮食类作物销售价格每年的增长率：', sp_growth)
print('蔬菜类作物销售价格每年的增长率：', vp_growth)
print('食用菌的销售价格每年的增长率：', mp_growth)
print('羊肚菌的销售价格每年下降幅度：', m1p_growth)

In [ ]:
# k = 0.5
# min_area_percent = 0.2
# max_plots = 4
# year = 2025
# list = [0.5, 0.2, 4, 2025]
# variables, binary_variables, actual_sales, excess_yield = create_variables(new_fields, crops)
# # 创建线性规划问题
# prob = pulp.LpProblem("Crop_Planting_Optimization", pulp.LpMaximize)
# profit = define_objective_function(variables, actual_sales, excess_yield, crops, new_fields, k)
# # 将目标函数添加到问题中
# prob += profit
# prob = add_constraints(prob, variables, binary_variables, actual_sales, 
#                        excess_yield, crops, new_fields, total_yield_2023, df2024_result, df2023_result,
#                        min_area_percent, max_plots)
# # 求解问题
# prob.solve()
# print("Status:", pulp.LpStatus[prob.status])
# # 更新new_fields
# update_new_fields(variables, new_fields)
# # 输出结果
# df_template = pd.read_excel('../excels/附件3/template.xlsx')
# output(df_template, year, new_fields)

In [ ]:
# 添加约束条件
# 地块面积限制
for field in new_fields:
    prob += pulp.lpSum([variables[(field.field_name, crop.crop_name, field.season)] for crop in crops]) <= field.field_area

# 作物种植限制
for field in new_fields:
    for crop in crops:
        if not check_crop_constraints(field, crop):
            prob += variables[(field.field_name, crop.crop_name, field.season)] == 0

# 重茬限制
for field in new_fields:
    for crop in crops:
        if not check_rotation_constraints(field, crop, df2023_result):
            prob += variables[(field.field_name, crop.crop_name, field.season)] == 0

# 实际销售量约束
for crop in crops:
    actual_yield = pulp.lpSum([
        variables[(field.field_name, crop.crop_name, field.season)] * get_yield(field, crop)
        for field in new_fields
    ])
    expected_sales = get_expected_sales(crop, total_yield_2023)
    prob += actual_sales[crop.crop_name] <= actual_yield
    prob += actual_sales[crop.crop_name] <= expected_sales

    # 超额部分约束
    prob += excess_yield[crop.crop_name] == actual_yield - expected_sales # 这里不是>=，应该是==？！
    prob += excess_yield[crop.crop_name] >= 0

# # 作物在单个地块某一季节种植面积不宜太小
# min_area_percent = 0.2
# for field in new_fields:
#     for crop in crops:
#         var = variables[(field.field_name, crop.crop_name, field.season)]
#         binary_var = binary_variables[(field.field_name, crop.crop_name, field.season)]
#         min_area = min_area_percent * field.field_area
#         prob += var >= min_area * binary_var
#         prob += var <= field.field_area * binary_var

# 作物种植不宜太分散
max_plots = 4
for crop in crops:
    for field_type in set(field.field_type for field in new_fields):
        if field_type not in ['普通大棚', '智慧大棚']:
            prob += pulp.lpSum([
                binary_variables[(field.field_name, crop.crop_name, field.season)]
                for field in new_fields if field.field_type == field_type
            ]) <= max_plots

In [ ]:

df2023_result = pd.read_excel('../excels/附件3/2023_result.xlsx')
# 去除列名末尾的空格。很奇怪
df2023_result.columns = [column.rstrip(' ') for column in df2023_result.columns]
# print(df2023_result.columns)
df2024_result = pd.read_excel('../excels/附件3/2024_result.xlsx')
# 去除列名末尾的空格。很奇怪
df2024_result.columns = [column.rstrip(' ') for column in df2024_result.columns]
# print(df2024_result.columns)

In [ ]:
list = [0.5, 0.2, 4, 2025]
df_last1_result = pd.read_excel('../excels/附件3/2023_result.xlsx')
df_last1_result.columns = [column.rstrip(' ') for column in df_last1_result.columns]
df_last2_result = pd.read_excel('../excels/附件3/2024_result.xlsx')
df_last2_result.columns = [column.rstrip(' ') for column in df_last2_result.columns]
optimize_planting_strategy(new_fields, crops, total_yield_2023, df2024_result, df2023_result, list)

## 以下暂时作废

In [ ]:
import pulp

# 假设 df1, df2023, df2023_result, total_yield, crops, new_fields 已经初始化

# 创建线性规划问题
prob = pulp.LpProblem("Crop_Planting_Optimization", pulp.LpMaximize)

# 创建决策变量
variables = {}
for field in new_fields:
    for crop in crops:
        var_name = f"{field.field_name}_{crop.crop_name}_{field.season}"
        variables[(field.field_name, crop.crop_name, field.season)] = pulp.LpVariable(var_name, lowBound=0)

# 输出每个变量的键值对
for varkey, varval in variables.items():
    print(varkey, varval)

('A1', '黄豆', '单季') A1_黄豆_单季
('A1', '黑豆', '单季') A1_黑豆_单季
('A1', '红豆', '单季') A1_红豆_单季
('A1', '绿豆', '单季') A1_绿豆_单季
('A1', '爬豆', '单季') A1_爬豆_单季
('A1', '小麦', '单季') A1_小麦_单季
('A1', '玉米', '单季') A1_玉米_单季
('A1', '谷子', '单季') A1_谷子_单季
('A1', '高粱', '单季') A1_高粱_单季
('A1', '黍子', '单季') A1_黍子_单季
('A1', '荞麦', '单季') A1_荞麦_单季
('A1', '南瓜', '单季') A1_南瓜_单季
('A1', '红薯', '单季') A1_红薯_单季
('A1', '莜麦', '单季') A1_莜麦_单季
('A1', '大麦', '单季') A1_大麦_单季
('A1', '水稻', '单季') A1_水稻_单季
('A1', '豇豆', '单季') A1_豇豆_单季
('A1', '刀豆', '单季') A1_刀豆_单季
('A1', '芸豆', '单季') A1_芸豆_单季
('A1', '土豆', '单季') A1_土豆_单季
('A1', '西红柿', '单季') A1_西红柿_单季
('A1', '茄子', '单季') A1_茄子_单季
('A1', '菠菜 ', '单季') A1_菠菜__单季
('A1', '青椒', '单季') A1_青椒_单季
('A1', '菜花', '单季') A1_菜花_单季
('A1', '包菜', '单季') A1_包菜_单季
('A1', '油麦菜', '单季') A1_油麦菜_单季
('A1', '小青菜', '单季') A1_小青菜_单季
('A1', '黄瓜', '单季') A1_黄瓜_单季
('A1', '生菜 ', '单季') A1_生菜__单季
('A1', '辣椒', '单季') A1_辣椒_单季
('A1', '空心菜', '单季') A1_空心菜_单季
('A1', '黄心菜', '单季') A1_黄心菜_单季
('A1', '芹菜', '单季') A1_芹菜_单季
('A1', '大白菜', '单季') A1_大白菜_单季
('A1

In [ ]:
print([variables[(field.field_name, crop.crop_name, field.season)] for crop in crops for field in new_fields])
# print([variables[(field.field_name, crop.crop_name, field.season)] * crop.crop_price for crop in crops for field in new_fields for crop in crops])
# print([total_yield[crop.crop_name] for crop in crops])

[A1_黄豆_单季, A2_黄豆_单季, A3_黄豆_单季, A4_黄豆_单季, A5_黄豆_单季, A6_黄豆_单季, B1_黄豆_单季, B2_黄豆_单季, B3_黄豆_单季, B4_黄豆_单季, B5_黄豆_单季, B6_黄豆_单季, B7_黄豆_单季, B8_黄豆_单季, B9_黄豆_单季, B10_黄豆_单季, B11_黄豆_单季, B12_黄豆_单季, B13_黄豆_单季, B14_黄豆_单季, C1_黄豆_单季, C2_黄豆_单季, C3_黄豆_单季, C4_黄豆_单季, C5_黄豆_单季, C6_黄豆_单季, D1_黄豆_单季, D1_黄豆_第一季, D1_黄豆_第二季, D2_黄豆_单季, D2_黄豆_第一季, D2_黄豆_第二季, D3_黄豆_单季, D3_黄豆_第一季, D3_黄豆_第二季, D4_黄豆_单季, D4_黄豆_第一季, D4_黄豆_第二季, D5_黄豆_单季, D5_黄豆_第一季, D5_黄豆_第二季, D6_黄豆_单季, D6_黄豆_第一季, D6_黄豆_第二季, D7_黄豆_单季, D7_黄豆_第一季, D7_黄豆_第二季, D8_黄豆_单季, D8_黄豆_第一季, D8_黄豆_第二季, E1_黄豆_第一季, E1_黄豆_第二季, E2_黄豆_第一季, E2_黄豆_第二季, E3_黄豆_第一季, E3_黄豆_第二季, E4_黄豆_第一季, E4_黄豆_第二季, E5_黄豆_第一季, E5_黄豆_第二季, E6_黄豆_第一季, E6_黄豆_第二季, E7_黄豆_第一季, E7_黄豆_第二季, E8_黄豆_第一季, E8_黄豆_第二季, E9_黄豆_第一季, E9_黄豆_第二季, E10_黄豆_第一季, E10_黄豆_第二季, E11_黄豆_第一季, E11_黄豆_第二季, E12_黄豆_第一季, E12_黄豆_第二季, E13_黄豆_第一季, E13_黄豆_第二季, E14_黄豆_第一季, E14_黄豆_第二季, E15_黄豆_第一季, E15_黄豆_第二季, E16_黄豆_第一季, E16_黄豆_第二季, F1_黄豆_第一季, F1_黄豆_第二季, F2_黄豆_第一季, F2_黄豆_第二季, F3_黄豆_第一季, F3_黄豆_第二季, F4_黄豆_第一季, F4_黄豆_第二季, A1_黑豆_单季, A2_黑豆_单季, A3_黑

In [ ]:
# 定义目标函数的各个部分
actual_yield = pulp.lpSum([variables[(field.field_name, crop.crop_name, field.season)] * 
                           get_yield(field.field_name, crop.crop_name) for field in new_fields for crop in crops])

# 收入部分：实际销售量 * 作物价格
revenue = pulp.lpSum([
    variables[(field.field_name, crop.crop_name, field.season)] * 
    crop.crop_price * min(get_total_yield_2023(crop.crop_name, total_yield_2023), actual_yield)
    for field in new_fields for crop in crops
])

# 成本部分：种植成本
cost = pulp.lpSum([
    variables[(field.field_name, crop.crop_name, field.season)] * get_cost(field, crop)
    for field in new_fields for crop in crops
])

# 超额部分处理：超额部分 * 作物价格 * 系数
excess_handling = pulp.lpSum([
    max(0, actual_yield - get_total_yield_2023(crop.crop_name, total_yield_2023)) * crop.crop_price * 0.5
    for field in new_fields for crop in crops
])

# 总利润 = 收入 - 成本 + 超额部分处理
profit = revenue - cost + excess_handling

# 将目标函数添加到问题中
prob += profit

TypeError: '>' not supported between instances of 'float' and 'LpAffineExpression'

In [ ]:


# 添加约束条件
# 地块面积限制
for field in new_fields:
    prob += pulp.lpSum([variables[(field.field_name, crop.crop_name, field.season)] for crop in crops]) <= field.field_area

# 作物种植限制
for field in new_fields:
    for crop in crops:
        if not check_crop_constraints(field, crop):
            prob += variables[(field.field_name, crop.crop_name, field.season)] == 0

# 重茬限制
for field in new_fields:
    for crop in crops:
        if not check_rotation_constraints(field, crop):
            prob += variables[(field.field_name, crop.crop_name, field.season)] == 0

# 豆类作物种植要求
for field in new_fields:
    prob += pulp.lpSum([variables[(field.field_name, crop.crop_name, field.season)] for crop in crops if crop.crop_type in ['粮食（豆类）', '蔬菜（豆类）']]) >= 1

# 求解问题
prob.solve()

# 打印结果
print("Status:", pulp.LpStatus[prob.status])
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)

# 更新 new_fields 列表中的种植情况
for field in new_fields:
    field.planted_crop = []
    for crop in crops:
        var_name = f"{field.field_name}_{crop.crop_name}_{field.season}"
        if variables[(field.field_name, crop.crop_name, field.season)].varValue > 0:
            field.planted_crop.append([crop.crop_name, variables[(field.field_name, crop.crop_name, field.season)].varValue])